In [56]:
import numpy as np
from scipy.spatial import Delaunay
from scipy.stats import norm
import open3d as o3d

In [57]:
# generate top surface coords
n = 30

mu1, sigma1 = 0, 1
mu2, sigma2 = 0, 1
x_lim = 3 * sigma1
z_lim = x_lim

x = np.linspace(-x_lim, x_lim, n)
z = np.linspace(-z_lim, z_lim, n)

X, Z = np.meshgrid(x, z)

Y = norm.pdf(X, mu1, sigma1) * norm.pdf(Z, mu2, sigma2) * 10
top_surface_matrix = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T
top_surface_matrix
# top_surface_matrix[:,2] = top_surface_matrix[:,2]*-1

array([[-3.00000000e+00,  1.96412803e-04, -3.00000000e+00],
       [-2.79310345e+00,  3.57632223e-04, -3.00000000e+00],
       [-2.58620690e+00,  6.23897149e-04, -3.00000000e+00],
       ...,
       [ 2.58620690e+00,  6.23897149e-04,  3.00000000e+00],
       [ 2.79310345e+00,  3.57632223e-04,  3.00000000e+00],
       [ 3.00000000e+00,  1.96412803e-04,  3.00000000e+00]],
      shape=(900, 3))

In [58]:
# zeroize top edges
for coord in top_surface_matrix:
    if coord[0] == -x_lim or coord[0] == x_lim or coord[2] == -z_lim or coord[2] == z_lim:
        coord[1] = 0

In [59]:
# offset top surface by sidewall height
sidewall_height = x_lim / 10
top_surface_matrix[:, 1] += sidewall_height

In [60]:
top_pcd = o3d.geometry.PointCloud()
top_pcd.points = o3d.utility.Vector3dVector(top_surface_matrix)
top_pcd.estimate_normals()
o3d.visualization.draw_geometries([top_pcd])

In [61]:
#TODO instead of creating the box layer by layer: create each wall layer by layer and save as an stl
# save all stls as temp files then combine

box_matrix = []

offset = sidewall_height / 10  # this should be based off sidewall height, which in turn should be based on the width of the box(limits are based on 250/250
current_offset = offset
for k in range(1, 10):  # number of layers
    layer = []
    for i in x:
        for j in z:
            if i == -x_lim or i == x_lim or j == -z_lim or j == z_lim:
                coord = np.array((i, 0, j))
                layer.append(coord)
    layer = np.array(layer)
    layer[:, 1] += current_offset
    box_matrix.append(layer)
    current_offset += offset

box_matrix = np.concat(box_matrix)

box_pcd = o3d.geometry.PointCloud()
box_pcd.points = o3d.utility.Vector3dVector(box_matrix)
box_pcd.estimate_normals()
o3d.visualization.draw_geometries([box_pcd])

In [62]:
bottom_layer = []
for i in x:
    for j in z:
        coord = np.array((i, 0, j))
        bottom_layer.append(coord)
bottom_layer = np.array(bottom_layer)

bottom_pcd = o3d.geometry.PointCloud()
bottom_pcd.points = o3d.utility.Vector3dVector(bottom_layer)
bottom_pcd.estimate_normals()
o3d.visualization.draw_geometries([bottom_pcd])

In [63]:
o3d.visualization.draw_geometries([bottom_pcd + box_pcd + top_pcd])

In [64]:
import openstl

top_triangles = Delaunay(top_surface_matrix[:, [0, 2]]).simplices

top_stl = openstl.convert.triangles(top_surface_matrix, top_triangles)

openstl.write('..\\out\\top.stl', top_stl)

True

In [65]:
box_triangles = Delaunay(box_matrix[:, [0, 2]]).simplices

box_stl = openstl.convert.triangles(box_matrix, box_triangles)

openstl.write('..\\out\\box.stl', box_stl)

True

In [66]:
bottom_triangles = Delaunay(bottom_layer[:, [0, 2]]).simplices

bottom_stl = openstl.convert.triangles(bottom_layer, bottom_triangles)

openstl.write('..\\out\\bottom.stl', bottom_stl)

True

In [ ]:
# 1. make a meshgrid for each of the 6 surfaces of the plot: some are (x,y), some are (x,z), some are (y,z)
# 2. Add the 3rd dimension to each grid and create a matrix: top surface requires y=norm.pdf, bottom y=0,
# 3. triangulate each surface
# 4. create a mesh of each surface and render with open3d
# 5. combine each mesh and deduplicate verticies
# 6. Validate stl and save to disk